# Previsão da ocorrência de diabetes.
>### Autor: [Alberto Oliveira Barbosa](https://www.linkedin.com/in/alberto-oliveira-barbosa)

## Objetivos do projeto:
### Desenvolver um modelo de previsão com ao menos <b>75%</b> de acerto sobre a ocorrência de diabetes, com o mínimo possivel de alterações nos dados originais

## Introdução
<p>Este é um projeto desenvolvido para fins de <b>estudo</b> e tem por objetivo testar a eficiência de alguns algoritmos de machine learning, e tentar chegar a porcentagem estabelecida sem que aja a necessidade de modificações e limpezas excessivas nos dados. </p>
<p>Por se tratar de um exercício de fixação não será feito nenhum tipo de ajuste mais específico nos inputs ou atributos dos algoritmos escolhidos, está é apenas uma experiência para verificar como os algoritmos se comportam.</p>
<p> Como o este é um problema típico de classificação será adotado a utilização os seguintes algoritmos:</p>
 - Naive Bayes
 - RandomForest
 - Regressão Logística



## Fonte de dados:
>[Pima Indians Diabetes Database - Kaggle](https://www.kaggle.com/uciml/pima-indians-diabetes-database/data)

## Importação das bibliotecas e configuração do notebook


In [231]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
% matplotlib inline

## Carga dos dados e primeiras visualizações

In [4]:
dados = pd.read_csv('pima-data.csv')

In [6]:
# Primeiros 5 registros
dados.head(5)

,num_preg,glucose_conc,diastolic_bp,thickness,insulin,bmi,diab_pred,age,skin,diabetes
0,6,148,72,35,0,33.6,0.627,50,1.3790,True
1,1,85,66,29,0,26.6,0.351,31,1.1426,False
2,8,183,64,0,0,23.3,0.672,32,0.0000,True
3,1,89,66,23,94,28.1,0.167,21,0.9062,False
4,0,137,40,35,168,43.1,2.288,33,1.3790,True


In [8]:
# Ultimos registros 
dados.tail(5)

,num_preg,glucose_conc,diastolic_bp,thickness,insulin,bmi,diab_pred,age,skin,diabetes
763,10,101,76,48,180,32.9,0.171,63,1.8912,False
764,2,122,70,27,0,36.8,0.340,27,1.0638,False
765,5,121,72,23,112,26.2,0.245,30,0.9062,False
766,1,126,60,0,0,30.1,0.349,47,0.0000,True
767,1,93,70,31,0,30.4,0.315,23,1.2214,False


In [26]:
# Variaveis preditoras 
dados.columns[:9]

Index(['num_preg', 'glucose_conc', 'diastolic_bp', 'thickness', 'insulin',
       'bmi', 'diab_pred', 'age', 'skin'],
      dtype='object')

In [27]:
# Atributo classe (target)
dados.columns[-1]

'diabetes'

## Análise dos dados

In [36]:
# verifica se possui valores nulos
dados.isnull().values.any()

False

In [35]:
# Verificar a correlação entre as variáveis
dados.corr()

,num_preg,glucose_conc,diastolic_bp,thickness,insulin,bmi,diab_pred,age,skin,diabetes
num_preg,1.000000,0.129459,0.141282,-0.081672,-0.073535,0.017683,-0.033523,0.544341,-0.081672,0.221898
glucose_conc,0.129459,1.000000,0.152590,0.057328,0.331357,0.221071,0.137337,0.263514,0.057328,0.466581
diastolic_bp,0.141282,0.152590,1.000000,0.207371,0.088933,0.281805,0.041265,0.239528,0.207371,0.065068
thickness,-0.081672,0.057328,0.207371,1.000000,0.436783,0.392573,0.183928,-0.113970,1.000000,0.074752
insulin,-0.073535,0.331357,0.088933,0.436783,1.000000,0.197859,0.185071,-0.042163,0.436783,0.130548
bmi,0.017683,0.221071,0.281805,0.392573,0.197859,1.000000,0.140647,0.036242,0.392573,0.292695
diab_pred,-0.033523,0.137337,0.041265,0.183928,0.185071,0.140647,1.000000,0.033561,0.183928,0.173844
age,0.544341,0.263514,0.239528,-0.113970,-0.042163,0.036242,0.033561,1.000000,-0.113970,0.238356
skin,-0.081672,0.057328,0.207371,1.000000,0.436783,0.392573,0.183928,-0.113970,1.000000,0.074752
diabetes,0.221898,0.466581,0.065068,0.074752,0.130548,0.292695,0.173844,0.238356,0.074752,1.000000


In [254]:
# dimensões dos dados
print('Os dados possuem %d observações e %d variáveis' % (dados.shape))

Os dados possuem 768 observações e 10 variáveis


In [93]:
# converte os valores para numeros 
dados['diabetes'] = dados['diabetes'].map({True:1,False:0})

In [90]:
# Cria uma tupla com o formato (quantidade de pessoas COM diabetes, percentual em relação ao total)
com_diabetes = (len(dados.loc[dados['diabetes'] == 1]), len(dados.loc[dados['diabetes'] == 1]) / len(dados['diabetes'])*100)

In [86]:
# Cria uma tupla com o formato (quantidade de pessoas SEM diabetes, percentual em relação ao total)
sem_diabetes = (len(dados.loc[dados['diabetes'] == 0]), len(dados.loc[dados['diabetes'] == 0]) / len(dados['diabetes'])*100)

In [257]:
print('Distribuição do atributo classe:')
print('%d pessoas desenvolveram diabetes (%0.2f%%)' % com_diabetes)
print('%d pessoas não desenvolveram diabetes (%0.2f%%)' % sem_diabetes)
print('De um total de %d registros' % dados.shape[0])

Distribuição do atributo classe:
268 pessoas desenvolveram diabetes (34.90%)
500 pessoas não desenvolveram diabetes (65.10%)
De um total de 768 registros


## Preparação dos dados para o modelo
### Divisão dos dados em treino e teste

In [106]:
# pegando o nome dos atributos preditores
preditoras = list(dados.columns[:9])

In [107]:
# pegando o nome do atributo classe
target = dados.columns[-1]
target = [target]

In [189]:
# criando os valores x e y para o modelo
x, y = dados[preditoras].values, dados[target].values

In [190]:
# divisão dos valores do data frame em treino e teste
x_treino, x_teste, y_treino, y_teste = train_test_split(x,y,test_size=0.3, random_state=20)

In [191]:
# verificar se a divisão ocorreu como esperado
print('Dados de X divididos em : %0.2f%% para treino e %0.2f%% teste' % (len(x_treino) / len(dados)*100, len(x_teste)/ len(dados)*100))
print('Dados de Y divididos em : %0.2f%% para treino e %0.2f%% teste' % (len(y_treino) / len(dados)*100, len(y_teste)/ len(dados)*100))

Dados de X divididos em : 69.92% para treino e 30.08% teste
Dados de Y divididos em : 69.92% para treino e 30.08% teste


### Tratamento de valores inválidos
Verificar a quantidade de valores 0 (zero) em cada coluna

In [192]:
for coluna in preditoras:
    print('A coluna %s possui %.0f valores inválidos' % (coluna, len(dados.loc[dados[coluna] == 0])))

A coluna num_preg possui 111 valores inválidos
A coluna glucose_conc possui 5 valores inválidos
A coluna diastolic_bp possui 35 valores inválidos
A coluna thickness possui 227 valores inválidos
A coluna insulin possui 374 valores inválidos
A coluna bmi possui 11 valores inválidos
A coluna diab_pred possui 0 valores inválidos
A coluna age possui 0 valores inválidos
A coluna skin possui 227 valores inválidos


In [258]:
# substituir os valores 0 pela média
imputer = Imputer(missing_values=0, strategy = 'mean', axis = 0)
x_treino = imputer.fit_transform(x_treino)
x_teste = imputer.fit_transform(x_teste)

## Criação e treinamento do modelo com Naive Bayes

In [194]:
# Criando o modelo
modelo = GaussianNB()

In [196]:
# treinando os dados
modelo.fit(x_treino, y_treino.ravel())

GaussianNB(priors=None)

## Medindo a precisão do modelo

In [208]:
# criando a previsão
previsao_naive = modelo.predict(x_teste)

In [217]:
# Testando a previsão com os dados com metrics
print('Precisão dos dados de %0.2f%% com Naive Bayes' % (metrics.accuracy_score(y_teste, previsao_naive)*100))

Precisão dos dados de 74.03% com Naive Bayes


In [210]:
#matriz de confusão
matriz = metrics.confusion_matrix(y_teste, previsao_naive, labels=[1,0])
matriz

array([[ 47,  31],
       [ 29, 124]])

In [241]:
# estatísticas do modelo
resumo = metrics.classification_report(y_teste,previsao_naive, labels=[1,0])
print('Resumo do processo de classificação\n\n',resumo)

Resumo do processo de classificação

              precision    recall  f1-score   support

          1       0.62      0.60      0.61        78
          0       0.80      0.81      0.81       153

avg / total       0.74      0.74      0.74       231



## Otimização do modelo, testes com outros algoritmos:
### RandomForest 

In [261]:
# criação do modelo
modelo_v2 = RandomForestClassifier(random_state=20)

In [205]:
# treino
modelo_v2.fit(x_treino, y_treino.ravel())

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=20, verbose=0, warm_start=False)

In [224]:
previsao_random = modelo_v2.predict(x_teste)

In [226]:
# Medidas de precisão
print('Precisão de %0.2f%% com RandomForest' % (metrics.accuracy_score(y_teste,previsao_random)*100))

Precisão de 74.89% com RandomForest


In [229]:
print('Matriz de confusão \n\n', metrics.confusion_matrix(y_teste, previsao_random, labels=[1,0]))

Matriz de confusão 

 [[ 41  37]
 [ 21 132]]


In [239]:
print('Resumo do processo de classificação:\n\n', metrics.classification_report(y_teste,previsao_random, labels=[1,0]))

Resumo do processo de classificação:

              precision    recall  f1-score   support

          1       0.66      0.53      0.59        78
          0       0.78      0.86      0.82       153

avg / total       0.74      0.75      0.74       231



### Regressão Logistica

In [246]:
#criação do modelo
modelo_v3 = LogisticRegression(C=0.7, random_state=20)

In [247]:
# treinamento
modelo_v3.fit(x_treino,y_treino.ravel())

LogisticRegression(C=0.7, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=20, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [248]:
previsao_rl = modelo_v3.predict(x_teste)

In [249]:
# Precisão
print('Precisão de %0.2f%% com Regressão Logistica' % (metrics.accuracy_score(y_teste,previsao_rl)*100))

Precisão de 75.32% com Regressão Logistica


In [250]:
print('Matriz de confusão:\n\n', metrics.confusion_matrix(y_teste, previsao_rl, labels=[1,0]))

Matriz de confusão:

 [[ 43  35]
 [ 22 131]]


In [251]:
print('Resumo do processo de classificação:\n\n', metrics.classification_report(y_teste, previsao_rl, labels=[1,0]))

Resumo do processo de classificação:

              precision    recall  f1-score   support

          1       0.66      0.55      0.60        78
          0       0.79      0.86      0.82       153

avg / total       0.75      0.75      0.75       231



## Conclusões

<p>Como este exercício tinha por objetivo apenas testar a eficiência dos algoritmos de machine learning e visualizar seu funcionamento, não foram adotadas modificações 

significativas nos dados, apenas aquelas específicas para o bom funcionamento dos mesmos, como: </p>

 - Conversão de variáveis categóricas do atributo classe (True, False) em números (1,0).
 - Substituição dos valores 0 (zero) dos atributos preditores por sua média, para que eles não afetem as previsões dos algoritmos.

<p>Após a carga inicial dos dados foi feita uma rápida visualização de como os dados estavam distribuidos, seus valores iniciais, finais, o total de observações e se existiam valores nulos.</p>

<p>Em seguida, durante o processo de exploração dos dados foi constatado que em algumas colunas existiam valores zerados que poderiam vir a influenciar nas previsões e o tratamento adotado para estes casos foi a sua substituição pela média</p>

<p>Outra peculiariedade encontrada nos dados foi a porcentagem de distribuição dos valores do atributo classe: <b>34.90% True e 65.10% False</b>,
para uma divisão entre dados de treino e teste o ideal seria valores próximos aos 50%, uma possível solução para isso sem que fosse feita algum tipo de alteração dos dados, e com isso fugindo à proposta inicial, seria a coleta de mais amostras, como este modelo está sendo produzido para fins de estudo, esta solução não foi adotada.</p>

<p>A correlação entre as variáveis foi verificada, mas para fins de simplificação foi utilizada todas as variáveis preditoras para o desenvolvimento do modelo.</p>

<p>Os dados foram divididos em: 70% para treino e 30% para teste, além do valor de random_state adotado ser 20 para a criação dos modelos (caso seja necessário reproduzir os resultados)</p>

<p>Com o cenário descrito acima foi possível observar as seguintes taxas de acerto nas previsões:</p>

-  74.03% com Naive Bayes

-  74.89% com RandomForest

-  75.32% com Regressão Logistica